In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow import keras
import matplotlib.pyplot as plt
from keras import applications
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [ ]:
data = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
data['label'] = data['label'].astype('str')
data.head()

In [ ]:
print(f'There are total {data.shape[0]} images in our train data.')

In [ ]:
with open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json', 'r') as file:
    labels = json.load(file)
    
labels

In [ ]:
TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

plt.figure(figsize=(16, 15))
df_sample = data.sample(12).reset_index(drop=True)
for i in range(9):
    plt.subplot(3, 3, i+1)
    img = cv2.imread(os.path.join(TRAIN_PATH, df_sample.image_id[i]))
    img = cv2.resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.axis('off')
    plt.imshow(img)
    plt.title(labels.get(df_sample.label[i]))
plt.tight_layout()
plt.show()

In [ ]:
sns.countplot(data.label.values)
plt.title('Bar distribution of class labels')
plt.show()

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    shear_range=20,
    zoom_range=0.2,
    height_shift_range=0.1,
    width_shift_range=0.1,
    validation_split=0.2
)

train_imagegen = train_datagen.flow_from_dataframe(
    data,
    directory='../input/cassava-leaf-disease-classification/train_images',
    x_col='image_id',
    y_col='label',
    subset='training',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    class_mode='categorical',
    batch_size=32
)

In [ ]:
valid_datagen = keras.preprocessing.image.ImageDataGenerator(validation_split=0.3)

valid_imagegen = valid_datagen.flow_from_dataframe(
    data,
    directory='../input/cassava-leaf-disease-classification/train_images',
    x_col='image_id',
    y_col='label',
    subset='validation',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    class_mode='categorical',
    batch_size=32
)

In [ ]:
conv_base = EfficientNetB3(include_top = False, weights = None,input_shape = (224, 224, 3))
model = conv_base.output
model = layers.GlobalAveragePooling2D()(model)
model = layers.Dense(5, activation = "softmax")(model)
model = models.Model(conv_base.input, model)

model.compile(optimizer = Adam(lr = 0.001),loss = "categorical_crossentropy",metrics = ["acc"])


In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

In [ ]:
history = model.fit_generator( train_imagegen,
                               epochs =10,
                               steps_per_epoch = (len(data)*0.8) // 32,
                               validation_data = valid_imagegen,
                               validation_steps = (len(data)*0.2) // 32,
                               callbacks = [early_stop, reduce_lr])

In [ ]:
plt.figure(figsize=(25,10))
plt.subplot(1, 2, 1)
plt.plot(history.history['acc'], 'b*-', label="train_acc")
plt.plot(history.history['val_acc'], 'r*-', label="val_acc")
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], 'b*-', label="train_acc")
plt.plot(history.history['val_loss'], 'r*-', label="val_acc")
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
sample = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
predict = []
for image in sample.image_id:
    img = keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/' + image)
    img = keras.preprocessing.image.img_to_array(img)
    img = keras.preprocessing.image.smart_resize(img, (224, 224))
    img = np.expand_dims(img, 0)
    prediction = model.predict(img)
    predict.append(np.argmax(prediction))

In [ ]:
submission = pd.DataFrame({'image_id': sample.image_id, 'label': predict})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False) 